### <p style="text-align:left"><span style="color:green">Importing Required `Libraries`</span></p>

In [1]:
# Data manipulation and analysis libraries:
import numpy as np
import pandas as pd

# Data visualization libraries:
import matplotlib.pyplot as plt
import seaborn as sns

# Other libraries:
#from tqdm import tqdm
from dotenv import load_dotenv
import warnings
import os

# Data science imports:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    BaggingRegressor,
)
from sklearn.model_selection import KFold
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
    median_absolute_error
)

# Configure pandas to display all columns:
pd.set_option('display.max_columns', None)

# Render figures directly in the notebook:
%matplotlib inline

# Render higher resolution images:
%config InlineBackend.figure_format = 'retina'

# Ignore warning messages:
warnings.filterwarnings('ignore')

### <p style="text-align:left"><span style="color:green">Reading the `CSV File`</span></p>

In [7]:
# Load environment variables from .env file
load_dotenv()

# Set the path to the directory where csv data is stored
DATA_PATH = os.getenv("DATA_PATH")

# Listing the different datasets on our data directory
files = os.listdir(DATA_PATH)
for file in files:
    print(file)

tweets dataset.csv


In [10]:
tweets_df = pd.read_csv('../data/tweets dataset.csv', encoding = "ISO-8859-1")